In [1]:
from Bio import SeqIO
from Bio.Alphabet import IUPAC
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from io import StringIO # Python 3

from Bio.SeqUtils import GC
from Bio.SeqUtils import MeltingTemp

import random
from datetime import datetime
from Levenshtein import distance 
from Levenshtein import hamming 
import numpy as np

import itertools

nucleotides = ['A', 'T', 'G', 'C']


In [23]:
def ATG_filter(sequence_list): 
    
    filtered=[]

    for sequence in sequence_list:
        if ('ATG' in sequence) == False:
            filtered.append(sequence)
                    
    return(filtered)

def AGG_filter(sequence_list): 
    
    filtered=[]

    for sequence in sequence_list:
        if ('AGG' in sequence) == False:
            filtered.append(sequence)
                    
    return(filtered)

def CC_GG_filter(sequence_list): 
    
    filtered=[]

    for sequence in sequence_list:
        if (('GG' in sequence) or ('CC' in sequence)) == False:
            filtered.append(sequence)
                    
    return(filtered)

def AAA_TTT_filter(sequence_list): #Get rid of polyA and polyT sequences
    
    filtered=[]

    for sequence in sequence_list:
        if (('AAA' in sequence) or ('TTT' in sequence)) == False:
            filtered.append(sequence)
                    
    return(filtered)


def AA_TT_filter(sequence_list): #Get rid of polyA and polyT sequences
    
    filtered=[]

    for sequence in sequence_list:
        if (('AA' in sequence) or ('TT' in sequence)) == False:
            filtered.append(sequence)
                    
    return(filtered)

def splice(barcodes,starts,ends):
    spliced = []
    
    for p in range(0,len(starts)):
        start = starts[p]
        end = ends[p]
        for barcode in barcodes:
            spliced_seq = barcode[start:end]
            spliced.append(spliced_seq)

    return spliced

def join_two_lists(barcodes_1,barcodes_2):
    joined = []
    
    for barcode_1 in barcodes_1:
        for barcode_2 in barcodes_2:
            joined_seq = barcode_1+barcode_2
            joined.append(joined_seq)
            
    return joined

def join_barcode(barcodes,seq):
    joined = []
    
    for barcode in barcodes:
        joined_seq = barcode+seq
        joined.append(joined_seq)

    return joined

def generate_all_combinations(sequence_length):
    
    result=set()

    for j in itertools.combinations_with_replacement(nucleotides, sequence_length):
        for k in itertools.permutations(j):
            result.add(''.join(k))
        
    return list(result)


def compute_orthogonality_simple(net):
    slide_distance = len(net[0])
    distances = []
    
    for nodeA in net:
        for nodeB in net:
            
            min_hamming = len(nodeA)+(len(nodeA));

            for slide in range(0,slide_distance):

                nodeA_extended = nodeA+("X"*slide_distance)
                nodeB_extended = ("X"*slide)+nodeB+("X"*(slide_distance-slide))

                current_hamming = hamming(nodeA_extended,nodeB_extended)

                if current_hamming < (min_hamming): #There are always 6 positions where Xs will align, so we need to subtract that from the score
                    min_hamming = current_hamming
        
            distances.append(min_hamming) #hamming distances of all nodes in network from candidate node
        
    edge_matrix = np.reshape(distances,[len(net),len(net)])
    
    return edge_matrix


def return_primer_list(net):
    
    primers = []
    for nodeA in net:
        for nodeB in net:
            primers.append(nodeA)
            
    return primers

def TM_filter_result(sequence_list,temp_target):
    result_filtered=set()

    for sequence in list(sequence_list):
        if (MeltingTemp.Tm_GC(sequence) > temp_target[0]):
            if MeltingTemp.Tm_GC(sequence) < temp_target[1]:
                result_filtered.add(sequence)
                
    return(list(result_filtered))

In [3]:
target_length = 17
barcodes = []

for record in SeqIO.parse('bc25mer240k.fasta',"fasta"):
    seq_string = str("%s" % (record.seq))
    barcodes.append(seq_string[0:target_length])



In [23]:
#Filter all barcodes to remove those containing GG or CC
barcodes = CC_GG_filter(barcodes)
print(len(barcodes))

#Filter all barcodes to remove those containing AA or TT
barcodes = AA_TT_filter(barcodes) 
print(len(barcodes))

barcodes = ATG_filter(barcodes)
print(len(barcodes)) #Filter for ATG

barcodes = AGG_filter(barcodes)
print(len(barcodes)) #Filter for AGG

barcodes = TM_filter_result(barcodes,[45,50])
print(len(barcodes)) #Filter for Tm


720
720
720
720
720


In [22]:
index_primers = list(set(splice(barcodes,[0,3,6,9],[3,6,9,12])))#non redundant set of primers with 3 bases
index_primers = CC_GG_filter(index_primers)
index_primers = AA_TT_filter(index_primers)
index_primers = ATG_filter(index_primers)

print(len(index_primers))

rbs_spacer = 'CGGTGG' #Avoid using A because of shine-dalgarno similarity

#Structure: Index-17bp-AGGTGG

35


In [13]:
#Network elimination implementation in python

def network_elimination(full_barcodes,network_size,threshold):
    
    networks = []
    c_iter =0
    walks=0
        
    while len(networks) < 1 and walks <200:
        walks+=1;
        #seed is fixed
        
        network = []
        random.seed(datetime.now()) #Change seed for random number
                
        print('Walk number ',walks)#seed is the universal barcode, no need to remove
        candidate_sequences = full_barcodes.copy() #Create new bucket of candidate sequences
        
        seed = candidate_sequences[random.randint(0,len(candidate_sequences)-1)] #set seed sequence
        print('Seed chosen: ',seed)#seed is the universal barcode, no need to remove
        
        candidate_sequences.remove(seed) #remove for consideration
        network.append(seed) #add to network

        while (len(network)<=network_size) and c_iter<10000: #Should try maxiter times before starting again from same seed 
            
            c_iter+=1
            
            if len(candidate_sequences) == 0:
                print('There are no more candidate sequences left')
                print(network,' did not meet minimum size')

            else:
                
                candidate = candidate_sequences[random.randint(0,len(candidate_sequences)-1)] 

                candidate_network = []#already includes seed
                candidate_network.append(seed)
                candidate_network.append(candidate) #no seed needed, seed is first sequence

                orthogonality = compute_orthogonality(candidate_network,20)
                minimum_orthogonality = (1-np.identity(len(candidate_network)))*threshold

                below_threshold = (orthogonality-minimum_orthogonality)<0  #Those that are true do not meet minimum threshold
                
                number_nonzero_elements = (np.size(candidate_network)*np.size(candidate_network)) - np.size(candidate_network)
                score = np.sum(orthogonality-minimum_orthogonality)/number_nonzero_elements #score is the mean value above the threshold

                if (score > 0) and (any(np.concatenate(below_threshold) == False)): 
                    print(candidate ,'(Network Score (Normalised):',score ,') was added')
                    network.append(candidate)
                    candidate_sequences.remove(candidate) #Remove from candidate list to avoid duplicate sequences 

            if len(network) > network_size:
                print(network)
                networks.append(network)
                
    return [networks,score]

    
    

In [7]:
#Compute orthogonality score between all nodes in a network

def compute_orthogonality(net,sequence_length):
    
    slide_distance = 12 #set slide distance manually
    
    distances = []
    
    for nodeA in net: #iterate through list
        for nodeB in net:
            
            min_hamming = len(nodeA)+(len(nodeA));

            for slide in range(0,slide_distance):

                nodeA_extended = nodeA+("X"*slide_distance)
                nodeB_extended = ("X"*slide)+nodeB+("X"*(slide_distance-slide))

                current_hamming = hamming(nodeA_extended,nodeB_extended)

                if current_hamming < (min_hamming): #There are always 6 positions where Xs will align, so we need to subtract that from the score
                    min_hamming = current_hamming
        
            distances.append(min_hamming) #hamming distances of all nodes in network from candidate node
        
    edge_matrix = np.reshape(distances,[len(net),len(net)])
    
    return edge_matrix

In [26]:
barcode_orthogonality_scores = []
barcode_orthogonality_networks = []
barcode_orthogonality_seeds = []

sequence_length = 26 #Length = gRNA minimum + extra bases
threshold=5;

#For every candidate barcode, evaluate its orthogonality in a network

for barcode in barcodes: # every candidate barcode (14bp)
    barcode_sequences = join_barcode(index_primers,barcode+rbs_spacer) #Full minimal sequence with 3bp orthogonal overhang
    
    #Construct fictional network
    candidate_network_specific_targets = splice(barcode_sequences,[0],[20]) #create network of all the PAM adjacent sequences which will be the actual CRISPR sites - we are only evaluating these with respect to each other
    
    potential_networks = network_elimination(candidate_network_specific_targets,10,2) #at least 2bp positions different

    #orthogonality_adjusted = compute_orthogonality(network,20)#-((1-np.identity(len(network))))
    #orthogonality_thresholded = orthogonality_adjusted>((1-np.identity(len(network)))*threshold)
    #score = np.sum(orthogonality_thresholded) #score is the mean value above the threshold

    if len(potential_networks) >= 1:
        barcode_orthogonality_networks.append(potential_networks[0])
        barcode_orthogonality_scores.append(potential_networks[1])
        barcode_orthogonality_seeds.append(barcode)
    

Walk number  1
Seed chosen:  CGTATAGCGCTGACGACTAG
GAGATAGCGCTGACGACTAG (Network Score (Normalised): 1.0 ) was added
GTGATAGCGCTGACGACTAG (Network Score (Normalised): 1.0 ) was added
GTCATAGCGCTGACGACTAG (Network Score (Normalised): 1.0 ) was added
TAGATAGCGCTGACGACTAG (Network Score (Normalised): 1.0 ) was added
ATCATAGCGCTGACGACTAG (Network Score (Normalised): 1.0 ) was added
GCGATAGCGCTGACGACTAG (Network Score (Normalised): 1.0 ) was added
GCAATAGCGCTGACGACTAG (Network Score (Normalised): 1.0 ) was added
ACGATAGCGCTGACGACTAG (Network Score (Normalised): 1.0 ) was added
GTAATAGCGCTGACGACTAG (Network Score (Normalised): 1.0 ) was added
ATAATAGCGCTGACGACTAG (Network Score (Normalised): 1.0 ) was added
['CGTATAGCGCTGACGACTAG', 'GAGATAGCGCTGACGACTAG', 'GTGATAGCGCTGACGACTAG', 'GTCATAGCGCTGACGACTAG', 'TAGATAGCGCTGACGACTAG', 'ATCATAGCGCTGACGACTAG', 'GCGATAGCGCTGACGACTAG', 'GCAATAGCGCTGACGACTAG', 'ACGATAGCGCTGACGACTAG', 'GTAATAGCGCTGACGACTAG', 'ATAATAGCGCTGACGACTAG']
Walk number  1
Seed chose

Seed chosen:  CAGTGCGCTGCACTAGTACT
GCTTGCGCTGCACTAGTACT (Network Score (Normalised): 1.0 ) was added
TGCTGCGCTGCACTAGTACT (Network Score (Normalised): 1.0 ) was added
TCTTGCGCTGCACTAGTACT (Network Score (Normalised): 1.0 ) was added
AGATGCGCTGCACTAGTACT (Network Score (Normalised): 1.0 ) was added
AGCTGCGCTGCACTAGTACT (Network Score (Normalised): 1.0 ) was added
ATCTGCGCTGCACTAGTACT (Network Score (Normalised): 1.0 ) was added
TGATGCGCTGCACTAGTACT (Network Score (Normalised): 1.0 ) was added
GTCTGCGCTGCACTAGTACT (Network Score (Normalised): 1.0 ) was added
ACTTGCGCTGCACTAGTACT (Network Score (Normalised): 1.0 ) was added
TCATGCGCTGCACTAGTACT (Network Score (Normalised): 1.0 ) was added
['CAGTGCGCTGCACTAGTACT', 'GCTTGCGCTGCACTAGTACT', 'TGCTGCGCTGCACTAGTACT', 'TCTTGCGCTGCACTAGTACT', 'AGATGCGCTGCACTAGTACT', 'AGCTGCGCTGCACTAGTACT', 'ATCTGCGCTGCACTAGTACT', 'TGATGCGCTGCACTAGTACT', 'GTCTGCGCTGCACTAGTACT', 'ACTTGCGCTGCACTAGTACT', 'TCATGCGCTGCACTAGTACT']
Walk number  1
Seed chosen:  CGATGAGCACG

TACAGAGTGCTCTCTGACGA (Network Score (Normalised): 1.0 ) was added
TGTAGAGTGCTCTCTGACGA (Network Score (Normalised): 1.0 ) was added
TGAAGAGTGCTCTCTGACGA (Network Score (Normalised): 1.0 ) was added
['CTGAGAGTGCTCTCTGACGA', 'TCAAGAGTGCTCTCTGACGA', 'ACAAGAGTGCTCTCTGACGA', 'AGCAGAGTGCTCTCTGACGA', 'TGCAGAGTGCTCTCTGACGA', 'TCTAGAGTGCTCTCTGACGA', 'ACTAGAGTGCTCTCTGACGA', 'TATAGAGTGCTCTCTGACGA', 'TACAGAGTGCTCTCTGACGA', 'TGTAGAGTGCTCTCTGACGA', 'TGAAGAGTGCTCTCTGACGA']
Walk number  1
Seed chosen:  TCGGCAGCGCTCACTCTGAT
CGTGCAGCGCTCACTCTGAT (Network Score (Normalised): 1.0 ) was added
CGCGCAGCGCTCACTCTGAT (Network Score (Normalised): 1.0 ) was added
ATAGCAGCGCTCACTCTGAT (Network Score (Normalised): 1.0 ) was added
GTAGCAGCGCTCACTCTGAT (Network Score (Normalised): 1.0 ) was added
CACGCAGCGCTCACTCTGAT (Network Score (Normalised): 1.0 ) was added
CTAGCAGCGCTCACTCTGAT (Network Score (Normalised): 1.0 ) was added
ATCGCAGCGCTCACTCTGAT (Network Score (Normalised): 1.0 ) was added
CGAGCAGCGCTCACTCTGAT (Net

GATAGTCTGTCACGACACTG (Network Score (Normalised): 1.0 ) was added
['CTGAGTCTGTCACGACACTG', 'GCAAGTCTGTCACGACACTG', 'AGTAGTCTGTCACGACACTG', 'AGAAGTCTGTCACGACACTG', 'TCTAGTCTGTCACGACACTG', 'ACAAGTCTGTCACGACACTG', 'TGAAGTCTGTCACGACACTG', 'ACTAGTCTGTCACGACACTG', 'GACAGTCTGTCACGACACTG', 'TGCAGTCTGTCACGACACTG', 'GATAGTCTGTCACGACACTG']
Walk number  1
Seed chosen:  GATACGCTGACGCGCATACA
TCAACGCTGACGCGCATACA (Network Score (Normalised): 1.0 ) was added
ATCACGCTGACGCGCATACA (Network Score (Normalised): 1.0 ) was added
ACGACGCTGACGCGCATACA (Network Score (Normalised): 1.0 ) was added
CGAACGCTGACGCGCATACA (Network Score (Normalised): 1.0 ) was added
ACAACGCTGACGCGCATACA (Network Score (Normalised): 1.0 ) was added
ATAACGCTGACGCGCATACA (Network Score (Normalised): 1.0 ) was added
TCGACGCTGACGCGCATACA (Network Score (Normalised): 1.0 ) was added
CTCACGCTGACGCGCATACA (Network Score (Normalised): 1.0 ) was added
AGCACGCTGACGCGCATACA (Network Score (Normalised): 1.0 ) was added
CGCACGCTGACGCGCATACA (Net

CATGAGTAGCGCACTGCGAT (Network Score (Normalised): 1.0 ) was added
GACGAGTAGCGCACTGCGAT (Network Score (Normalised): 1.0 ) was added
GATGAGTAGCGCACTGCGAT (Network Score (Normalised): 1.0 ) was added
['TCGGAGTAGCGCACTGCGAT', 'GTCGAGTAGCGCACTGCGAT', 'AGTGAGTAGCGCACTGCGAT', 'AGCGAGTAGCGCACTGCGAT', 'CGAGAGTAGCGCACTGCGAT', 'ATCGAGTAGCGCACTGCGAT', 'CTCGAGTAGCGCACTGCGAT', 'ATAGAGTAGCGCACTGCGAT', 'CATGAGTAGCGCACTGCGAT', 'GACGAGTAGCGCACTGCGAT', 'GATGAGTAGCGCACTGCGAT']
Walk number  1
Seed chosen:  GTGGCGATCTACTCTGCATC
TCAGCGATCTACTCTGCATC (Network Score (Normalised): 1.0 ) was added
TGCGCGATCTACTCTGCATC (Network Score (Normalised): 1.0 ) was added
TACGCGATCTACTCTGCATC (Network Score (Normalised): 1.0 ) was added
CACGCGATCTACTCTGCATC (Network Score (Normalised): 1.0 ) was added
TATGCGATCTACTCTGCATC (Network Score (Normalised): 1.0 ) was added
TCTGCGATCTACTCTGCATC (Network Score (Normalised): 1.0 ) was added
AGCGCGATCTACTCTGCATC (Network Score (Normalised): 1.0 ) was added
CGCGCGATCTACTCTGCATC (Net

...

GCATCTGAGACGTGCGTGAT (Network Score (Normalised): 1.0 ) was added
GCTTCTGAGACGTGCGTGAT (Network Score (Normalised): 1.0 ) was added
GTATCTGAGACGTGCGTGAT (Network Score (Normalised): 1.0 ) was added
GTGTCTGAGACGTGCGTGAT (Network Score (Normalised): 1.0 ) was added
GATTCTGAGACGTGCGTGAT (Network Score (Normalised): 1.0 ) was added
CTATCTGAGACGTGCGTGAT (Network Score (Normalised): 1.0 ) was added
CTGTCTGAGACGTGCGTGAT (Network Score (Normalised): 1.0 ) was added
['TGCTCTGAGACGTGCGTGAT', 'ACTTCTGAGACGTGCGTGAT', 'CATTCTGAGACGTGCGTGAT', 'ACGTCTGAGACGTGCGTGAT', 'GCATCTGAGACGTGCGTGAT', 'GCTTCTGAGACGTGCGTGAT', 'GTATCTGAGACGTGCGTGAT', 'GTGTCTGAGACGTGCGTGAT', 'GATTCTGAGACGTGCGTGAT', 'CTATCTGAGACGTGCGTGAT', 'CTGTCTGAGACGTGCGTGAT']
Walk number  1
Seed chosen:  GTCCGCGTGCTGTAGTAGTG
TGACGCGTGCTGTAGTAGTG (Network Score (Normalised): 1.0 ) was added
TCACGCGTGCTGTAGTAGTG (Network Score (Normalised): 1.0 ) was added
TAGCGCGTGCTGTAGTAGTG (Network Score (Normalised): 1.0 ) was added
ACTCGCGTGCTGTAGTAGTG (Net

GTGCTGCTATCAGCTGCGTC (Network Score (Normalised): 1.0 ) was added
TACCTGCTATCAGCTGCGTC (Network Score (Normalised): 1.0 ) was added
TAGCTGCTATCAGCTGCGTC (Network Score (Normalised): 1.0 ) was added
GAGCTGCTATCAGCTGCGTC (Network Score (Normalised): 1.0 ) was added
CAGCTGCTATCAGCTGCGTC (Network Score (Normalised): 1.0 ) was added
CGCCTGCTATCAGCTGCGTC (Network Score (Normalised): 1.0 ) was added
GATCTGCTATCAGCTGCGTC (Network Score (Normalised): 1.0 ) was added
CACCTGCTATCAGCTGCGTC (Network Score (Normalised): 1.0 ) was added
['ACACTGCTATCAGCTGCGTC', 'CTGCTGCTATCAGCTGCGTC', 'GACCTGCTATCAGCTGCGTC', 'GTGCTGCTATCAGCTGCGTC', 'TACCTGCTATCAGCTGCGTC', 'TAGCTGCTATCAGCTGCGTC', 'GAGCTGCTATCAGCTGCGTC', 'CAGCTGCTATCAGCTGCGTC', 'CGCCTGCTATCAGCTGCGTC', 'GATCTGCTATCAGCTGCGTC', 'CACCTGCTATCAGCTGCGTC']
Walk number  1
Seed chosen:  GTAGTGTGCTCTGTAGTCAC
CAGGTGTGCTCTGTAGTCAC (Network Score (Normalised): 1.0 ) was added
CACGTGTGCTCTGTAGTCAC (Network Score (Normalised): 1.0 ) was added
TCTGTGTGCTCTGTAGTCAC (Net

TGCCAGCGTATCGTCTCGTC (Network Score (Normalised): 1.0 ) was added
['CATCAGCGTATCGTCTCGTC', 'TCACAGCGTATCGTCTCGTC', 'GTGCAGCGTATCGTCTCGTC', 'ATCCAGCGTATCGTCTCGTC', 'AGCCAGCGTATCGTCTCGTC', 'AGACAGCGTATCGTCTCGTC', 'GCACAGCGTATCGTCTCGTC', 'ATACAGCGTATCGTCTCGTC', 'TGACAGCGTATCGTCTCGTC', 'GTACAGCGTATCGTCTCGTC', 'TGCCAGCGTATCGTCTCGTC']
Walk number  1
Seed chosen:  TCGCTACGTAGAGAGCACGT
CTACTACGTAGAGAGCACGT (Network Score (Normalised): 1.0 ) was added
GATCTACGTAGAGAGCACGT (Network Score (Normalised): 1.0 ) was added
AGCCTACGTAGAGAGCACGT (Network Score (Normalised): 1.0 ) was added
ATACTACGTAGAGAGCACGT (Network Score (Normalised): 1.0 ) was added
GTACTACGTAGAGAGCACGT (Network Score (Normalised): 1.0 ) was added
CACCTACGTAGAGAGCACGT (Network Score (Normalised): 1.0 ) was added
CATCTACGTAGAGAGCACGT (Network Score (Normalised): 1.0 ) was added
GTCCTACGTAGAGAGCACGT (Network Score (Normalised): 1.0 ) was added
CGACTACGTAGAGAGCACGT (Network Score (Normalised): 1.0 ) was added
CGCCTACGTAGAGAGCACGT (Net

AGCAGAGTCTCAGCGCGTAC (Network Score (Normalised): 1.0 ) was added
['CTGAGAGTCTCAGCGCGTAC', 'GCTAGAGTCTCAGCGCGTAC', 'ACAAGAGTCTCAGCGCGTAC', 'TGCAGAGTCTCAGCGCGTAC', 'TGAAGAGTCTCAGCGCGTAC', 'AGAAGAGTCTCAGCGCGTAC', 'GATAGAGTCTCAGCGCGTAC', 'GCAAGAGTCTCAGCGCGTAC', 'ACTAGAGTCTCAGCGCGTAC', 'GACAGAGTCTCAGCGCGTAC', 'AGCAGAGTCTCAGCGCGTAC']
Walk number  1
Seed chosen:  TGAGCTAGAGCGTATCGTCA
CATGCTAGAGCGTATCGTCA (Network Score (Normalised): 1.0 ) was added
ACGGCTAGAGCGTATCGTCA (Network Score (Normalised): 1.0 ) was added
GATGCTAGAGCGTATCGTCA (Network Score (Normalised): 1.0 ) was added
GTGGCTAGAGCGTATCGTCA (Network Score (Normalised): 1.0 ) was added
ACTGCTAGAGCGTATCGTCA (Network Score (Normalised): 1.0 ) was added
CACGCTAGAGCGTATCGTCA (Network Score (Normalised): 1.0 ) was added
GAGGCTAGAGCGTATCGTCA (Network Score (Normalised): 1.0 ) was added
CAGGCTAGAGCGTATCGTCA (Network Score (Normalised): 1.0 ) was added
GCGGCTAGAGCGTATCGTCA (Network Score (Normalised): 1.0 ) was added
CTGGCTAGAGCGTATCGTCA (Net

CTCCTCGCTCTGCACTAGAC (Network Score (Normalised): 1.0 ) was added
CGCCTCGCTCTGCACTAGAC (Network Score (Normalised): 1.0 ) was added
GTCCTCGCTCTGCACTAGAC (Network Score (Normalised): 1.0 ) was added
TGCCTCGCTCTGCACTAGAC (Network Score (Normalised): 1.0 ) was added
GACCTCGCTCTGCACTAGAC (Network Score (Normalised): 1.0 ) was added
GTGCTCGCTCTGCACTAGAC (Network Score (Normalised): 1.0 ) was added
GTACTCGCTCTGCACTAGAC (Network Score (Normalised): 1.0 ) was added
TGACTCGCTCTGCACTAGAC (Network Score (Normalised): 1.0 ) was added
['ACTCTCGCTCTGCACTAGAC', 'CTGCTCGCTCTGCACTAGAC', 'CTACTCGCTCTGCACTAGAC', 'CTCCTCGCTCTGCACTAGAC', 'CGCCTCGCTCTGCACTAGAC', 'GTCCTCGCTCTGCACTAGAC', 'TGCCTCGCTCTGCACTAGAC', 'GACCTCGCTCTGCACTAGAC', 'GTGCTCGCTCTGCACTAGAC', 'GTACTCGCTCTGCACTAGAC', 'TGACTCGCTCTGCACTAGAC']


In [27]:
best_network = barcode_orthogonality_networks[np.argmax(barcode_orthogonality_scores)]
best_score = barcode_orthogonality_scores[np.argmax(barcode_orthogonality_scores)]
best_barcode = barcode_orthogonality_seeds[np.argmax(barcode_orthogonality_scores)]

print(best_network,'Has Network Score:',best_score, 'has seed ',best_barcode)

[['CGTATAGCGCTGACGACTAG', 'GAGATAGCGCTGACGACTAG', 'GTGATAGCGCTGACGACTAG', 'GTCATAGCGCTGACGACTAG', 'TAGATAGCGCTGACGACTAG', 'ATCATAGCGCTGACGACTAG', 'GCGATAGCGCTGACGACTAG', 'GCAATAGCGCTGACGACTAG', 'ACGATAGCGCTGACGACTAG', 'GTAATAGCGCTGACGACTAG', 'ATAATAGCGCTGACGACTAG']] Has Network Score: 1.0 has seed  ATAGCGCTGACGACTAG


In [34]:
final_barcode_index = best_network[0] #First remove from array

#Reconstruct all barcodes from targets
final_barcode = best_barcode[0:target_length]
final_barcode_index_primers = splice(final_barcode_index[:],[0],[3])#Index is first 3bp of wach of the best network entries
final_barcode_sequences = join_barcode(final_barcode_index_primers,final_barcode+rbs_spacer) #Full minimal sequence with 3bp orthogonal overhang

print(final_barcode_sequences)
#Construct Sequence RECORD Object from sequences
from Bio import SeqIO
from Bio.Alphabet import IUPAC
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

sequence_strings = final_barcode_sequences
upstream_overhang = "taatgattagtagttagtaa"
downstream_overhang = "ATTAAAGAGGAGAAATACTAG"

with open("final_barcode_sequences_3bp_minimal.fasta", "w") as output_handle:
    for i in range(0,len(sequence_strings)):
        name_constructor = 'barcode_'+str(i+1)
        simple_seq = Seq(sequence_strings[i])
        simple_seq_r = SeqRecord(simple_seq,id=name_constructor)
        SeqIO.write(simple_seq_r, output_handle, "fasta")
        
with open("final_barcode_sequences_3bp_minimal_with_gibson_overhangs.fasta", "w") as output_handle:
    for i in range(0,len(sequence_strings)):
        name_constructor = 'barcode_'+str(i+1)
        simple_seq = Seq(upstream_overhang+ sequence_strings[i]+downstream_overhang)
        simple_seq_r = SeqRecord(simple_seq,id=name_constructor,description="Barcode with Gibson Overhangs")
        SeqIO.write(simple_seq_r, output_handle, "fasta")


['CGTATAGCGCTGACGACTAGCGGTGG', 'GAGATAGCGCTGACGACTAGCGGTGG', 'GTGATAGCGCTGACGACTAGCGGTGG', 'GTCATAGCGCTGACGACTAGCGGTGG', 'TAGATAGCGCTGACGACTAGCGGTGG', 'ATCATAGCGCTGACGACTAGCGGTGG', 'GCGATAGCGCTGACGACTAGCGGTGG', 'GCAATAGCGCTGACGACTAGCGGTGG', 'ACGATAGCGCTGACGACTAGCGGTGG', 'GTAATAGCGCTGACGACTAGCGGTGG', 'ATAATAGCGCTGACGACTAGCGGTGG']


In [36]:
from Bio import SeqIO
from Bio.Alphabet import IUPAC
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from io import StringIO # Python 3

#Read in existing file and create sequence list

sequences_trimmed = []
from Bio import SeqIO
records = SeqIO.parse("mbar_all_barcode_sequences_3bp_minimal_with_gibson_overhangs.fasta", "fasta")
for record in records:
    sequences_trimmed.append(str(record.seq))
print(sequences_trimmed)

#Recover barcodes from file
sequences_trimmed = splice(sequences_trimmed,[20],[len(sequences_trimmed[0])-21])



['taatgattagtagttagtaaCGTATAGCGCTGACGACTAGCGGTGGATTAAAGAGGAGAAATACTAG', 'taatgattagtagttagtaaGAGATAGCGCTGACGACTAGCGGTGGATTAAAGAGGAGAAATACTAG', 'taatgattagtagttagtaaGTGATAGCGCTGACGACTAGCGGTGGATTAAAGAGGAGAAATACTAG', 'taatgattagtagttagtaaGTCATAGCGCTGACGACTAGCGGTGGATTAAAGAGGAGAAATACTAG', 'taatgattagtagttagtaaTAGATAGCGCTGACGACTAGCGGTGGATTAAAGAGGAGAAATACTAG', 'taatgattagtagttagtaaATCATAGCGCTGACGACTAGCGGTGGATTAAAGAGGAGAAATACTAG', 'taatgattagtagttagtaaGCGATAGCGCTGACGACTAGCGGTGGATTAAAGAGGAGAAATACTAG', 'taatgattagtagttagtaaGCAATAGCGCTGACGACTAGCGGTGGATTAAAGAGGAGAAATACTAG', 'taatgattagtagttagtaaACGATAGCGCTGACGACTAGCGGTGGATTAAAGAGGAGAAATACTAG', 'taatgattagtagttagtaaGTAATAGCGCTGACGACTAGCGGTGGATTAAAGAGGAGAAATACTAG', 'taatgattagtagttagtaaATAATAGCGCTGACGACTAGCGGTGGATTAAAGAGGAGAAATACTAG']


In [61]:
pEvolvR_upstream_overhang = "Gcccttggtacaatgctagc"
pEvolvR_downstream_overhang = "gttttagagctagaaatagcaag"

guide_RNAs_specific = splice(sequences_trimmed[:],[0],[20])#first 20bp of sequence strings = guide 1
guide_RNA_universal = splice(sequences_trimmed[:],[3],[len(sequences_trimmed[0])-3])#first 20bp of sequence strings = guide 1

guide_RNAs_all = []
guide_RNA_names = ["mbarU_gRNA"]
guide_RNAs_all.append(guide_RNA_universal[0]) #append universal guide

for i in range(0,len(guide_RNAs_specific)):
    guide_RNAs_all.append(guide_RNAs_specific[i]) #append all specific guides
    guide_RNA_names.append("mbarS"+str(i+1)+"_gRNA")

print(guide_RNAs_all)
    
with open("minimal_gRNA_sequences_3bp.fasta", "w") as output_handle:
    for i in range(0,len(guide_RNAs_all)):
        name_constructor = guide_RNA_names[i]
        
        simple_seq = Seq(guide_RNAs_all[i])
        
        #Python will prepend G to guides that do not start with G or C
        if (simple_seq[0] == "G"):
            simple_seq = Seq(guide_RNAs_all[i])
        else:
            simple_seq = Seq("G"+guide_RNAs_all[i])

        simple_seq_r = SeqRecord(simple_seq,id=name_constructor,description="gRNA")
        SeqIO.write(simple_seq_r, output_handle, "fasta")

with open("minimal_gRNA_sequences_3bp_with_gibson_overhangs.fasta", "w") as output_handle:
    for i in range(0,len(guide_RNAs_all)):
        name_constructor = guide_RNA_names[i]
        gRNA_seq = Seq(guide_RNAs_all[i])

        #Python will prepend G to guides that do not start with G
        if (gRNA_seq[0] == "G"):
            simple_seq = Seq(pEvolvR_upstream_overhang+ guide_RNAs_all[i]+pEvolvR_downstream_overhang)
        else:
            simple_seq = Seq(pEvolvR_upstream_overhang+ "G"+guide_RNAs_all[i]+pEvolvR_downstream_overhang)
        
        print(simple_seq)
            
        simple_seq_r = SeqRecord(simple_seq,id=name_constructor,description="gRNA with Gibson Overhangs")
        SeqIO.write(simple_seq_r, output_handle, "fasta")
        
    print()

['ATAGCGCTGACGACTAGCGG', 'CGTATAGCGCTGACGACTAG', 'GAGATAGCGCTGACGACTAG', 'GTGATAGCGCTGACGACTAG', 'GTCATAGCGCTGACGACTAG', 'TAGATAGCGCTGACGACTAG', 'ATCATAGCGCTGACGACTAG', 'GCGATAGCGCTGACGACTAG', 'GCAATAGCGCTGACGACTAG', 'ACGATAGCGCTGACGACTAG', 'GTAATAGCGCTGACGACTAG', 'ATAATAGCGCTGACGACTAG']
GcccttggtacaatgctagcGATAGCGCTGACGACTAGCGGgttttagagctagaaatagcaag
GcccttggtacaatgctagcGCGTATAGCGCTGACGACTAGgttttagagctagaaatagcaag
GcccttggtacaatgctagcGAGATAGCGCTGACGACTAGgttttagagctagaaatagcaag
GcccttggtacaatgctagcGTGATAGCGCTGACGACTAGgttttagagctagaaatagcaag
GcccttggtacaatgctagcGTCATAGCGCTGACGACTAGgttttagagctagaaatagcaag
GcccttggtacaatgctagcGTAGATAGCGCTGACGACTAGgttttagagctagaaatagcaag
GcccttggtacaatgctagcGATCATAGCGCTGACGACTAGgttttagagctagaaatagcaag
GcccttggtacaatgctagcGCGATAGCGCTGACGACTAGgttttagagctagaaatagcaag
GcccttggtacaatgctagcGCAATAGCGCTGACGACTAGgttttagagctagaaatagcaag
GcccttggtacaatgctagcGACGATAGCGCTGACGACTAGgttttagagctagaaatagcaag
GcccttggtacaatgctagcGTAATAGCGCTGACGACTAGgttttagagctagaaatagcaag
Gc